In [11]:
library(ggplot2)
library(gganimate)
library(rnaturalearth)
library(rnaturalearthdata)
library(dplyr)
library(sf)
library(gifski)
library(IRdisplay)

Warning message:
"package 'gifski' was built under R version 4.4.1"


In [3]:
security_incidents_clean <- read.csv("../data/processed/security_incidents_clean.csv")

In [4]:
head(security_incidents_clean)

,Incident.ID,Year,Month,Country.Code,Country,Region,District,City,UN,INGO,...,Means.of.attack,Attack.context,Location,Motive,Actor.type,Actor.name,Details,Verified,Source,Group
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1997,1,KH,Cambodia,Banteay Meanchey,Unknown,Unknown,0,0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1 ICRC national staff killed while working in Banteay Meanchey province.,Archived,Archived,Asia
2,2,1997,1,RW,Rwanda,Northern,Musanze,Ruhengeri,0,4,...,Shooting,Raid,Office/compound,Unknown,Unknown,Unknown,"3 INGO international (Spanish) staff killed, 1 INGO international (US) staff gravely wounded during armed raid on INGO compound in Ruhengeri, Rwanda.",Archived,Archived,Africa
3,3,1997,2,TJ,Tajikistan,Unknown,Unknown,Unknown,4,0,...,Kidnapping,Unknown,Unknown,Unknown,Unknown,Unknown,"3 UN national staff, 1 UN international (Nigerian) staff, 1 ICRC international staff and 1 ICRC national staff kidnapped along with other (not included) UN staff, journalist and Tajik govt rep; released 48 hours later.",Archived,Archived,Asia
4,4,1997,2,SO,Somalia,Lower Juba,Kismayo,Kismayo,0,1,...,Unknown,Unknown,Unknown,Political,Non-state armed group: Regional,Al-Itihaad al-Islamiya,1 INGO international staff killed by Al ittihad militia in Kismayo.,Archived,Archived,Africa
5,5,1997,2,RW,Rwanda,Kigali,Kigali,Kigali,1,0,...,Shooting,Individual attack,Unknown,Political,Unknown,Unknown,1 UN national staff shot and killed in Kigali Feb 14.,Archived,Archived,Africa
6,7,1997,5,CD,DR Congo,Unknown,Unknown,Unknown,0,0,...,Unknown,Combat/Crossfire,Unknown,Incidental,Non-state armed group: National,Alliance of Democratic Forces for the Liberation of Congo-Zaire (ADFL),10 NRCS staff first aid workers killed in fighting between Zairean troops and rebels from the Alliance of Democratic Forces for the Liberation of Congo-Zaire (ADFL).,Archived,Archived,Africa


In [7]:
colnames(security_incidents_clean)


[1] "Incident.ID"              "Year"                    
 [3] "Month"                    "Country.Code"            
 [5] "Country"                  "Region"                  
 [7] "District"                 "City"                    
 [9] "UN"                       "INGO"                    
[11] "ICRC"                     "NRCS.and.IFRC"           
[13] "NNGO"                     "Other"                   
[15] "Nationals.killed"         "Nationals.wounded"       
[17] "Nationals.kidnapped"      "Total.nationals"         
[19] "Internationals.killed"    "Internationals.wounded"  
[21] "Internationals.kidnapped" "Total.internationals"    
[23] "Total.killed"             "Total.wounded"           
[25] "Total.kidnapped"          "Total.affected"          
[27] "Gender.Male"              "Gender.Female"           
[29] "Gender.Unknown"           "Means.of.attack"         
[31] "Attack.context"           "Location"                
[33] "Motive"                   "Actor.type"              
[35] "Actor.name"               "Details"                 
[37] "Verified"                 "Source"                  
[39] "Group"

In [8]:
# Group by Country and Year, sum Total.affected
country_year_data <- security_incidents_clean |>
  group_by(Country, Year) |>
  summarise(TotalAffected = sum(Total.affected, na.rm = TRUE), .groups = "drop") |>
  filter(!is.na(TotalAffected) & TotalAffected > 0)


In [9]:
# Load world shapefile
world <- ne_countries(scale = "medium", returnclass = "sf")

# Join your summarized data with map data
world_data <- left_join(world, country_year_data, by = c("name" = "Country"))

In [10]:
p <- ggplot(world_data) +
  geom_sf(fill = "gray90", color = "white") +
  geom_sf(aes(size = TotalAffected), color = "red", alpha = 0.6) +
  scale_size_continuous(range = c(1, 20), breaks = c(10, 50, 100, 200)) +
  labs(
    title = 'Year: {closest_state}',
    size = 'Total Affected'
  ) +
  theme_minimal() +
  transition_states(Year, transition_length = 2, state_length = 1) +
  ease_aes('linear')
